In [9]:
import pandas as pd
import numpy as np
import pickle
import h5py
from pickle import UnpicklingError
import keras
import os
import sys

try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
    print("Está no google colab")
except:
    IN_COLAB = False
    print("Não está no google colab")


Não está no google colab


In [ ]:
##Local do diretorio
if IN_COLAB:
    diretorio = '/content/drive/Othercomputers/Meu-laptop/binary_prediction_poisonous_mushrooms/modelos'
else:
    diretorio = 'modelos'
##Pegando os amodelos salvos
selecionando_modelos = ['modelo_base_random_forest.pkl', 'modelo_base_gradient_boost.pkl', 'modelo_base_cat_boost.pkl', 'modelo_base_redes_neural.pkl', ## Modelos base
                       'random_forest_base_ven.pkl', 'gradiente_boost_base_ven.pkl', 'cat_boost_base_ven.pkl', 'redes_neural_base_ven.pkl', ## Modelos Calibrados com VennAbersCalibrator
                        'modelo_cat_boost.pkl','modelo_random_forest.pkl','modelo_redes_neural.h5',  # Modelos tunado
                        'redes_neural_tunado_ven.pkl', 'random_forest_tunado_ven.pkl', 'cat_boost_tunado_ven.pkl'] # Modelos tunado com VennAbersCalibrator
dic_modelos = {}
for index, model in enumerate(selecionando_modelos):
    with open(diretorio+'/'+ model, 'rb') as f:
        if '_base_' in selecionando_modelos[index]:
            dic_modelos[selecionando_modelos[index].split('.pkl')[0]] = pickle.load(f)
        elif '_ven' in selecionando_modelos[index]:
            dic_modelos[selecionando_modelos[index].split('.pkl')[0]] = pickle.load(f)
        else:
            try:
                dic_modelos[selecionando_modelos[index].split('.pkl')[0] + '_tunado'] = pickle.load(f)
            except UnpicklingError:
                dic_modelos[selecionando_modelos[index].split('.h5')[0] + '_tunado'] = keras.models.load_model(diretorio + '/modelo_redes_neural.h5')
#

In [ ]:
import h5py
# Salve o dicionário em um arquivo HDF5
with h5py.File(diretorio +'/modelos.h5', 'w') as f:
    for key, model in dic_modelos.items():
        # Serializa cada modelo usando pickle
        serialized_model = pickle.dumps(model)
        # Salva o modelo serializado como um dataset binário no arquivo HDF5
        f.create_dataset(key, data=np.void(serialized_model))

In [11]:
if IN_COLAB:
    sys.path.append("/content/drive/Othercomputers/Meu-laptop/binary_prediction_poisonous_mushrooms/modelos")
    arquivos = os.listdir("/content/drive/Othercomputers/Meu-laptop/binary_prediction_poisonous_mushrooms/modelos")
    diretorio = '/content/drive/Othercomputers/Meu-laptop/binary_prediction_poisonous_mushrooms/modelos'
else:
    sys.path.append("modelos")
    arquivos = os.listdir("modelos")

    diretorio = 'modelos'
os.path.exists(diretorio)

True

In [13]:
modelos_carregados = {}

# Abra o arquivo HDF5 e carregue os modelos
with h5py.File(diretorio+'/modelos.h5', 'r') as f:
    for key in f.keys():
        # Carrega o modelo serializado do dataset HDF5
        serialized_model = f[key][()]
        # Deserializa o modelo usando pickle
        model = pickle.loads(serialized_model.tobytes())
        # Adiciona o modelo ao dicionário
        modelos_carregados[key] = model

OSError: Unable to synchronously open file (truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048)